<h2>Wrapper combining OpenAI, Melissa, Yelp</h2>

In [21]:
pip install xmltodict openai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: xmltodict in c:\python3.11\lib\site-packages (0.13.0)




[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import urllib.request, urllib.parse, os, json, xmltodict, asyncio, requests
from openai import AsyncOpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [24]:
async def openai_wrapper(location, preferences = ""):
    # Get a list of 10 locations form OpenAI api
    openai_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    chat_completion = await openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": f"I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}. Generate a list of 10 attraction objects and return each attraction name and entire address"}
        ],
    )
    return json.loads(chat_completion.choices[0].message.content)["attractions"]

In [25]:
async def yelp_wrapper(location, term = "attractions", radius = 20000, sort_by = "best_match", limit = 20):
    # Get a list of 20 locations form Yelp api
    params = {
        "location": location,
        "term": term,
        "radius": radius,
        "sort_by": sort_by,
        "limit": limit
    }
    yelp_url = f"https://api.yelp.com/v3/businesses/search?{urllib.parse.urlencode(params)}"
    yelp_header = {
        "accept": "application/json",
        "Authorization": f"Bearer {os.getenv('YELP_API_KEY')}"
    }
    response = await asyncio.to_thread(requests.get, yelp_url, headers = yelp_header)
    if response.status_code == 200:
        yelp_locations = response.json()
    else:
        response.raise_for_status()
    return yelp_locations
    

In [30]:
async def api_wrapper(location, preferences=""):
    # Run both openai_wrapper and yelp_wrapper concurrently
    openai_result, yelp_result = await asyncio.gather(
        openai_wrapper(location, preferences),
        yelp_wrapper(location, preferences)
    )
    return {"OpenAI": openai_result, "Yelp": yelp_result}

In [32]:
print(await api_wrapper("American High School, Fremont", "technology museum"))

{'Yelp': {'businesses': [{'id': 'Z6gkivXc4B_eG5oj4OgaxQ', 'alias': 'computer-history-museum-mountain-view', 'name': 'Computer History Museum', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/TFjFch8js4Gz33QW5biwOQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/computer-history-museum-mountain-view?adjust_creative=spgT8FBiP7sP9-7-q3Wy6Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=spgT8FBiP7sP9-7-q3Wy6Q', 'review_count': 357, 'categories': [{'alias': 'museums', 'title': 'Museums'}], 'rating': 4.5, 'coordinates': {'latitude': 37.41428183615418, 'longitude': -122.07740497882996}, 'transactions': [], 'location': {'address1': '1401 N Shoreline Blvd', 'address2': '', 'address3': '', 'city': 'Mountain View', 'zip_code': '94043', 'country': 'US', 'state': 'CA', 'display_address': ['1401 N Shoreline Blvd', 'Mountain View, CA 94043']}, 'phone': '+16508101010', 'display_phone': '(650) 810-1010', 'distance': 17624.413409156747}, {'id': 'YVt1YFiLeldQrNXC